In [ ]:
import numpy as np
import os
import sys
from scipy.fftpack import dct, idct
from scipy.optimize import minimize
sys.path.append(os.path.join('..','UTILS'))

# cosamp function is available at https://github.com/avirmaux/CoSaMP :
#    ITERATIVE SIGNAL RECOVERY FROM INCOMPLETE AND INACCURATE SAMPLES
# copy function from ipynb into cosamp_fn.py and place in UTILS folder
from cosamp_fn import cosamp

import holoviews as hv; hv.extension('bokeh')

In [ ]:
## Generate signal, DCT of signal

n   = 4096                       # points in high resolution signal
t   = np.linspace(0,1,n)
x   = np.cos(2 * 97 * np.pi * t) + np.cos(2 * 777 * np.pi * t)
xt  = np.fft.fft(x)              # Fourier transformed signal
PSD = xt * np.conj(xt) / n       # Power spectral density

In [ ]:
## Randomly sample signal
p    = 128 # num. random samples, p = n/32
perm = np.floor(np.random.rand(p) * n).astype(int)
y    = x[perm]

In [ ]:
## Solve compressed sensing problem
Psi   = dct(np.identity(n))                            # Build Psi
Theta = Psi[perm,:]                                    # Measure rows of Psi

s      = cosamp(Theta,y,10,epsilon=1.e-10,max_iter=10) # CS via matching pursuit
xrecon = idct(s)                                       # reconstruct full signal

xtrecon = np.fft.fft(xrecon,n)                         # computes the (fast) discrete fourier transform
PSDrecon = xtrecon * np.conj(xtrecon)/n                # Power spectrum (how much power in each freq)

In [ ]:
## Plot
time_window = np.array([1024,1280])/4096
freq        = np.arange(n)
L           = int(np.floor(n/2))

h=\
hv.Curve((t,        x),                         "t",         "amplitude").opts(xlim=(time_window[0],time_window[1]), padding=0.05)*\
hv.Scatter((perm/n, y)                                                  ).opts(color='red', size=5)+\
hv.Curve((freq[0:L],PSD.real[0:L]),     "Frequency", "Magnitude") +\
hv.Curve((t,        xrecon),                    "t",         "amplitude").opts(xlim=(time_window[0],time_window[1]), padding=0.05)+\
hv.Curve((freq[0:L],PSDrecon.real[:L]), "Frequency", "Magnitude")
h.cols(2)

In [ ]:
## L1-Minimization using SciPy
def L1_norm(x):
    return np.linalg.norm(x,ord=1)

constr = ({'type': 'eq', 'fun': lambda x:  Theta @ x - y})
x0     = np.linalg.pinv(Theta) @ y 
res    = minimize(L1_norm, x0, method='SLSQP',constraints=constr)
s      = res.x